# Predict

In [1]:
# Install necessary modules.
!echo "Start KerasCV"
!pip install -q /kaggle/input/kerasv3-lib-ds/keras_cv-0.8.2-py3-none-any.whl --no-deps
!echo "End KerasCV"

!echo "Start TensorFlow"
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!echo "End TensorFlow"

!echo "Start Keras"
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl --no-deps
!echo "End Keras"

Start KerasCV
End KerasCV
Start TensorFlow
End TensorFlow
Start Keras
End Keras


In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch

import joblib
import keras_cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

from config_hms_hbac import Config
from data_handler_hms_hbac import DataHandler
from trainer_hms_hbac import Trainer

2024-03-18 02:16:45.086959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 02:16:45.087011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 02:16:45.088383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...


In [3]:
# https://keras.io/api/keras_cv/models/tasks/image_classifier/
config = Config(pretrained_model="resnet50_v2_imagenet",
                image_size=[400, 300],
                epochs=100,
                batch_size=32,
                lr_mode="cos")

In [4]:
trainer = Trainer(config)

trainer.set_model()
trainer.compile_model()
trainer.show_model_summary()

Attaching 'config.json' from model 'keras/resnetv2/keras/resnet50_v2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/resnetv2/keras/resnet50_v2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/resnetv2/keras/resnet50_v2_imagenet/2' to your Kaggle notebook...


Model: "image_classifier_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, None, 3)     │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ res_net50v2_backbone            │ (None, None, None, 2048)  │ 23,564,800 │
│ (ResNetV2Backbone)              │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ avg_pool                        │ (None, 2048)              │          0 │
│ (GlobalAveragePooling2D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ predictions (Dense)             │ (None, 6)                 │     12,294 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 23,577,094 (89.94 MB)

 Trainable params: 23,531,654 (89.77 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [5]:
model_path = "/kaggle/input/hms_hbac/keras/test/2/20240316_resnet50_v2_imagenet.keras"
trainer.model.load_weights(model_path)

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:394: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 350 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [6]:
data_handler = DataHandler(config)
data_handler.prepare_for_test()

  0%|          | 0/1 [00:00<?, ?it/s]

2024-03-18 02:17:34.731126: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 02:17:34.731180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 02:17:34.732771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
preds = trainer.model.predict(data_handler.test_ds)

pred_df = data_handler.test_df[["eeg_id"]].copy()
target_cols = [x.lower()+'_vote' for x in config.class_names]
pred_df[target_cols] = preds.tolist()

sub_df = pd.read_csv(f'{data_handler.base_path}/sample_submission.csv')
sub_df = sub_df[["eeg_id"]].copy()
sub_df = sub_df.merge(pred_df, on="eeg_id", how="left")
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.278258,0.13996,0.019572,0.094801,0.308265,0.159143


In [8]:
import glob 
for f in glob.glob('*'):
    if not f.startswith('subm'):
        !rm -rf {f}